In [10]:
import time
from datetime import datetime

import pandas as pd
from database import Database
from sklearn.pipeline import Pipeline
from sqlalchemy.dialects.postgresql import insert
from ssrs import BookingData
from transformers import BookingDataEncoder, BookingDataReadCsv

In [11]:
db = Database()

reservation = db.gwg_reservation

/home/jovyan/work/database.py:34: SAWarning: Did not recognize type 'geometry' of column 'geom'
  Table(


In [12]:
date_fr = "02/02/2024"
date_to = "02/03/2024"

In [13]:
pipeline = Pipeline(
    [
        ("csv_reader", BookingDataReadCsv()),
        ("data_encoder", BookingDataEncoder()),
    ]
)

In [15]:
dfs = []

for destination in ["DU", "OM"]:
    booking_data = BookingData(destination, date_fr, date_to, 3).get()

    df = pipeline.fit_transform(booking_data)
    dfs.append(df)

In [16]:
records = pd.concat(dfs, axis=0, ignore_index=True)

In [17]:
records.empty

False

In [9]:
if records is not None and not records.empty:
    chunk_size = 1000
    for i in range(0, len(records), chunk_size):
        chunk = records.iloc[i : i + chunk_size]

        stmt = insert(reservation).values(chunk.to_dict(orient="records"))
        stmt = stmt.on_conflict_do_update(
            constraint="gwg_reservation_pkey", set_=dict(stmt.excluded)
        )

        with db.engine.begin() as connection:
            connection.execute(stmt)
else:
    print("empty")

empty


In [76]:
import io
from datetime import date
from ftplib import FTP

import numpy as np
import pandas as pd
from config import Config

In [77]:
ftp_server = Config.FTP_SERVER
ftp_username = Config.FTP_USERNAME
ftp_password = Config.FTP_PASSWORD
current_date = date.today().strftime("%Y_%m_%d")

In [78]:
ftp = FTP(ftp_server)
ftp.login(ftp_username, ftp_password)
ftp.cwd("group_report/")

'250 CWD command successful.'

In [79]:
csv_files = [x for x in ftp.nlst() if ("UAE" in x or "OM" in x) and current_date in x]
csv_files

['OM_Marginreport_GWG_2024_02_03_080014.csv',
 'OM_Marginreport_GWG_BLANK_2024_02_03_080014.csv',
 'UAE_Marginreport_GWG_2024_02_03_080014.csv',
 'UAE_Marginreport_GWG_BLANK_2024_02_03_080014.csv',
 'YT_OM_Marginreport_GWG_2024_02_03_080325.csv',
 'YT_UAE_Marginreport_GWG_2024_02_03_055007.csv']

In [80]:
data = io.BytesIO()

In [81]:
data.seek(0)

0

In [82]:
ftp.retrbinary("RETR " + "UAE_Marginreport_GWG_BLANK_2024_02_03_080014.csv", data.write)

'226 Transfer complete.'

In [83]:
data.seek(0)

0

In [84]:
df = pd.read_csv(
    data,
    sep=";",
    usecols=[0, 1, 5],
    header=None,
    names=["bkg_ref", "operator_code", "operator_price"],
    thousands=",",
    dtype={
        "bkg_ref": str,
        "operator_code": str,
        "operator_price": float,
    },
)

In [85]:
df = (
    df.groupby(["bkg_ref", "operator_code"])["operator_price"]
    .sum()
    .reset_index(name="operator_price")
)

In [86]:
df.shape

(3006, 3)

In [87]:
df.to_csv("raw_cost.csv", index=False)